# Object Detection - Image 




### Import our libraries and building useful functions for object detection

In [19]:
import os
import cv2
import numpy as np
import cvlib as cv
import matplotlib.pyplot as plt
from cvlib.object_detection import draw_bbox

In [10]:
# This function remove the components that are smaller than praticular threshold

def keepLargeComponents(image, treshold):
    
    frame = np.zeros(image.shape) < 0 # boolean array
    unique_labels = np.unique(image.flatten()) # find out every unique value that is actually a label 
    
    for label in unique_labels:
        if label == 0: # background
            pass
        else:
            img = (image == label) # save the component
            if treshold < np.sum(img):
                frame = frame | img # save all the components
                
    return np.float32(255*frame)

In [11]:
# This function save sequence on the disk

def saveSequence(caputre, frame_counter, frameSequence, outPutPath):
    
    treshold = frameSequence
    
    if len(caputre) < treshold:
        pass
    else:
        counter = 1
        for frame in caputre:
            image_name = str(frame_counter)+'_'+ str(counter)+'.jpg'
            finalPath = os.path.join(outPutPath, image_name)
            bbox, labels, conf = cv.detect_common_objects(frame)
            frame = draw_bbox(frame, bbox, labels, conf)
            cv2.imwrite(finalPath, frame)
            
            counter += 1

### Build a path to load and save our data

In [12]:
# This path is for loading images to detection
#images_path = (r'C:\Users') # add file(iamges) location 

# This path is loaction for the saved images 
# outPutPath = (r'C:\Users') # add folder location to save 

In [10]:
# This path is for loading images to detection
images_path = (r'C:\Users\Admin\Downloads\Udemy\UdemyCv\Data\GroundtruthSeq\RawImages')

# This path is loaction for the saved images 
outPutPath = (r'C:\Users\Admin\Downloads\Udemy\UdemyCv\Data\smart')

## Object detection function

In [ ]:
foregroundModel = cv2.createBackgroundSubtractorMOG2()

# save sequence of 5 frames
frameSequence = 5
frame_counter = 0
capture = []
index = []

for image_name in os.listdir(images_path):
    
    frame_counter += 1 # tell us in wich frame we are right now
    frame = cv2.imread(os.path.join(images_path, image_name))
    frame = cv2.resize(frame, dsize=(600, 400))
    
    foreground_mask = foregroundModel.apply(frame)
    
    # Create a matrix of frame and foreground mask
    frame_mask = np.zeros(frame.shape, np.uint8)
    frame_mask[:,:,0] = foreground_mask
    frame_mask[:,:,1] = foreground_mask
    frame_mask[:,:,2] = foreground_mask
    foreground_frame = np.hstack((frame, frame_mask))
    
    # Reduce noises
    structuring_element = cv2.getStructuringElement(cv2.MORPH_RECT,(3,3))
    foreground_mask = cv2.morphologyEx(np.float32(foreground_mask), cv2.MORPH_OPEN, structuring_element)
    
    # Find out connected components and keep only the large components
    num_labels, image_labels = cv2.connectedComponents(np.array(0<foreground_mask, np.uint8))
    foreground_mask = keepLargeComponents(image_labels, treshold=1000) # return components larger than threshold
    
    # If it's not a background..
    if 0 < np.sum(foreground_mask):
        index.append(frame_counter) # in wich frame the change is apears 
        capture.append(frame)    # adding the last frame 
        
    # If there are more then 2 frames and the change happens in consecutive frames..
    if 2 <= len(index) and index[-2]+1 < index[-1]:
        saveSequence(capture, frame_counter, frameSequence, outPutPath)
        capture = []
        index = []
        
    # Plot frame
    cv2.imshow('frame', foreground_frame)
    
    # Key to stop the video
    key = cv2.waitKey(5) & 0xff
    if key == 27:
        break
        
saveSequence(capture, frame_counter, frameSequence, outPutPath)
cv2.destroyAllWindows()

## Show Image Detection

In [20]:
# This function load images from folder and show image detection

def showDetection(outPutPath):
    
    for image_name in os.listdir(outPutPath):
        frame = cv2.imread(os.path.join(outPutPath, image_name))
        frame = cv2.resize(frame, dsize=(600,400))
        cv2.imshow('Display', frame)
        
        key = cv2.waitKey(30) & 0xff
        if key == 27:
            break
            
    cv2.destroyAllWindows()

In [ ]:
showDetection(outPutPath)